In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import defaultdict

In [7]:
def l1_norm(x1, x2):
    return np.sum(np.abs(x1 - x2), axis=1)

def l2_norm(x1, x2):
    return np.sqrt(np.sum(np.square(x1 - x2), axis=1))
 
def f(x, y):
    return y if 1 > x > 0 else 0  

def simple(x):
    return x

def cosine(x):
    return f(x, math.pi / 4 * math.cos(x / 2 * math.pi))

#TODO: def gauss, kd-tree, tk-fold

def f1_score(pred, real):
    metrics = calculate_metrics(pred, real)
    precision = metrics["TP"] / (metrics["P"])
    recall = metrics["TP"] / (metrics["TP"] + metrics["FN"])
    return 2 * precision * recall / (precision + recall)

def calculate_metrics(pred, real):
    metrics = {
        "TP": 0,
        "FP": 0,
        "TN": 0,
        "FN": 0
    }
    for (p, r) in zip(pred, real):
        if r == 1:
            metrics["TP" if p == 1 else "FN"] += 1
        if r == 0:
            metrics["FP" if p == 1 else "TN"] += 1
    metrics["P"] = metrics["TP"] + metrics["FP"]
    metrics["N"] = metrics["TN"] + metrics["FN"]
    return metrics

def mode(arr):
    app = defaultdict(int)
    m, c = 0, 0
    for e in arr:
        app[e] += 1
        if app[e] > c:
            c = app[e]
            m = e
    return m       


class KNN:
    def __init__(self, metric=l2_norm, kernel=simple):
        self.metric = metric
        self.kernel = kernel

    def train(self, X, y):
        self.Xtr = X
        self.ytr = y

    def predict(self, k, X, kernel=cosine):
        num_tests = X.shape[0]
        y_pred = np.zeros(num_tests, dtype=self.ytr.dtype)
        for i in range(num_tests):
            ranges = self.kernel(self.metric(self.Xtr, X[i, :]))
            best_k = np.take(self.ytr, np.argpartition(ranges, k, axis=0)[:k])
            y_pred[i] = mode(best_k)
        return y_pred

In [8]:
def best_k(X, y, metric=l2_norm, folds=8, kernel=simple):
    sample = np.c_[X, y]
    res = {}
    np.random.shuffle(sample)
    arrays = np.array_split(sample, folds)
    for fold in range(0, folds): 
        X_val = arrays[fold][:, : -1]
        Y_val = arrays[fold][:, -1]
        X_tr = np.delete(arrays, fold)
        Y_tr = np.delete(arrays, fold)
        X_tr, Y_tr = np.concatenate(X_tr)[:, : -1], np.concatenate(Y_tr)[:, -1]
        for k in range(2, 55):
            knn = KNN(metric, kernel)
            knn.train(X_tr, Y_tr)
            res[k] = res.get(k, 0) + f1_score(knn.predict(k, X_val), Y_val) / folds
    return res

In [13]:
chips = pd.read_csv('chips.txt', header=None)
chips = chips.sample(frac=1).reset_index(drop=True)
chips[3] = chips[0] ** 2 + chips[1] ** 2
train_to = 90

X, y = chips.loc[:train_to, [0, 1, 3]].as_matrix(), chips.loc[:train_to, [2]].as_matrix()
for c in chips.as_matrix():
    if c[2] == 1:
        plt.plot(c[0], c[1], 'rx')
    else:
        plt.plot(c[0], c[1], 'bo')
plt.show()

In [11]:
def argmax(d):
    v = list(d.values())
    k = list(d.keys())
    return k[v.index(max(v))]

ks = best_k(X, y)
k = argmax(ks)
knn = KNN(l2_norm)
knn.train(X, y)
X_test, y_test = chips.loc[train_to:, [0, 1, 3]].as_matrix(), chips.loc[train_to:, [2]].as_matrix()
print(f1_score(knn.predict(k, X_test), y_test))
plt.plot(list(ks.keys()), list(ks.values()))
plt.show()

0.7058823529411764


In [13]:
ks = best_k(X, y, metric=l1_norm)
k2 = argmax(ks)
knn = KNN(l1_norm)
knn.train(X,y)
print(f1_score(knn.predict(k2, X_test), y_test))
plt.show(plt.plot(list(ks.keys()), list(ks.values())))

0.8387096774193549
